In [78]:
import clip
import pandas as pd
import os
import torch 
import numpy as np
import string
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertForSequenceClassification
import json
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import clip
from transformers import CLIPProcessor, CLIPModel
import tqdm
from transformers import CLIPTextConfig, CLIPTextModel
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm 





device = "cuda" if torch.cuda.is_available() else "cpu"
print(device,"device")



#Note : launch the bert etc with the new mapping csv

cuda device


In [12]:
summary_bert_path = "/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Summaries/export_summary_Bert.csv"
summary_bert = pd.read_csv(summary_bert_path)


summary_bert_ingredients_path="/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Summaries/export_summary_bert_with_ingredients.csv"
summary_bert_ingredients=pd.read_csv(summary_bert_ingredients_path)


images_path="/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Food Images/Food Images/"

data_dir = "ComputerVision_Data"
images_dir = "Food Images/Food Images"

Creation of a clean dataset with the same number of images and text 

In [55]:
import pandas as pd
import pandas as pd

recipe = summary_bert["Summary"].tolist()
print(type(recipe))
print("number of sumary recipe ",len(recipe))

#print(recipe[:2])

images = os.listdir(os.path.join(data_dir, images_dir))
#print(type(images))
print("number of images ",len(images))

liste=summary_bert["Image_Name"].tolist()
liste = [item + ".jpg" for item in liste]

#print(liste[:5])


images = [image for image in images if os.path.basename(image) in liste ]  
print("after adding some filtering we have number of images ",len(images))

images = [image[:-4] for image in images]


#Create a CSV with only the images and the recipe who fit together

# Read the CSV file into a DataFrame
csv_path = 'ComputerVision_Data/Food Ingredients and Recipe Dataset with Image Name Mapping.csv'
csv_path2= 'ComputerVision_Data/Mapping.csv'
df = pd.read_csv(csv_path)

# Filter the DataFrame based on the "Image_Name" column
df_filtered = df[df['Image_Name'].isin(images)]

# Save the filtered DataFrame back to the CSV file
df_filtered.to_csv(csv_path2, index=False)

print(len(df_filtered))


<class 'list'>
number of sumary recipe  13463
number of images  13582
after adding some filtering we have number of images  13463
13463


In [64]:
dataset = "/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Mapping.csv"
dataset = pd.read_csv(dataset)

Instructions=dataset["Instructions"].tolist()
Ingredients=dataset["Ingredients"].tolist()
images=dataset["Image_Name"]

path_folder_image= "/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Food Images/Food Images/"
images = [os.path.join(path_folder_image, image) for image in images]
images = [item + ".jpg" for item in images]

print(images[:5])
print(Instructions[:5])
print(Ingredients[:5])


['/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Food Images/Food Images/miso-butter-roast-chicken-acorn-squash-panzanella.jpg', '/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Food Images/Food Images/crispy-salt-and-pepper-potatoes-dan-kluger.jpg', '/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Food Images/Food Images/thanksgiving-mac-and-cheese-erick-williams.jpg', '/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Food Images/Food Images/italian-sausage-and-bread-stuffing-240559.jpg', '/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Food Images/Food Images/newtons-law-apple-bourbon-cocktail.jpg']
['Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour.\nMeanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half int

data processing on the text

In [67]:
to_supress = string.punctuation + "—" + "–" + "()"

for i in range(len(Ingredients)): 
    temp = Ingredients[i].translate(str.maketrans("", "",to_supress))
    temp= ''.join([char for char in temp if not char.isdigit()])
    Ingredients[i]=temp.replace("Tbsp", "").replace("½", "").replace("¾", "").replace("lb", "").replace("tsp", "").replace("⅓", "").replace("  ", "").replace("¼", "")
    
    #A remplacer quand on aura trouvé la bonne manière de rallonger le context length
    if len(Ingredients)>77:
        Ingredients[i]=Ingredients[i][:77]

print(Ingredients[:5])


['whole chicken kosher salt divided plus moresmall acorn squash about total fin', ' large egg whitespound new potatoes aboutinch in diameterteaspoons kosher sal', ' cup evaporated milkcup whole milk garlic powder onion powder smoked paprika ', 'to pound round Italian loaf cut into inch cubescupstablespoons olive oil divi', ' teaspoon dark brown sugarteaspoon hot water oz bourbonoz fresh lemon juicete']


In [82]:
# Latest Update : 18 July 2022, 09:55 GMT+7

# TO ADD :
# Gradient Checkpointing
# Filter out bias from weight decay
# Decaying learning rate with cosine schedule
# Half-precision Adam statistics
# Half-precision stochastically rounded text encoder weights were used

#BATCH_SIZE must larger than 1

model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training

BATCH_SIZE=8
EPOCH=2
optimizer=optim.Adam(model.parameters(), lr=0.001)


class image_title_dataset(Dataset):
    def __init__(self, list_image_path,list_txt):

        self.image_path = list_image_path
        self.title  = clip.tokenize(list_txt) #you can tokenize everything at once in here(slow at the beginning), or tokenize it in the training loop.

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        image = preprocess(Image.open(self.image_path[idx])) # Image from PIL module
        title = self.title[idx]
        return image,title
    


# use your own data
list_image_path = images 
list_txt = Ingredients
dataset = image_title_dataset(list_image_path,list_txt)
train_dataloader = DataLoader(dataset,batch_size = BATCH_SIZE) #Define your own dataloader

#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 


if device == "cpu":
  model.float()
else :
  clip.model.convert_weights(model) # Actually this line is unnecessary since clip by default already on float16

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()


# add your own code to track the training progress.
for epoch in tqdm(range(EPOCH)):
  for batch in train_dataloader :
      optimizer.zero_grad()

      images,texts = batch 
      print("image shape", images.shape)
      print("text shape",texts.shape)
    
      images= images.to(device)
      texts = texts.to(device)
    
      logits_per_image, logits_per_text = model(images, texts)

      ground_truth = torch.arange(len(images),dtype=torch.long,device=device)

      total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
      total_loss.backward()
      if device == "cpu":
         optimizer.step()
      else : 
        convert_models_to_fp32(model)
        optimizer.step()
        clip.model.convert_weights(model)


torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"/users/eleves-b/2022/hanna.mergui/Computer-Vision/ComputerVision_Data/Checkpoints/checkpoint_number_1") 

AttributeError: 'Tensor' object has no attribute 'read'